In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import re
from sklearn.feature_selection import mutual_info_regression
from wordcloud import WordCloud


# Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 1000)

# Load the data
filepath = r"D:\#Data\Assessments\Axion Ray\Task 2.xlsx"
df = pd.read_excel(filepath)

# Check basic info
print(f"Dataset shape: {df.shape}")
print("\nData types:")
print(df.dtypes)

# Check for missing values
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame(
    {"Missing Values": missing_values, "Percentage": missing_percent}
)
print("\nMissing values summary:")
print(
    missing_df[missing_df["Missing Values"] > 0].sort_values(
        "Percentage", ascending=False
    )
)

# Basic statistics for numerical columns
print("\nNumerical columns summary:")
print(df.describe())

# Check unique values for categorical columns
categorical_cols = df.select_dtypes(include=["object"]).columns
for col in categorical_cols[:5]:  # Showing first 5 to keep output manageable
    print(f"\nUnique values in {col}: {df[col].nunique()}")
    print(df[col].value_counts().head(3))

Dataset shape: (100, 73)

Data types:
VIN                                          object
TRANSACTION_ID                                int64
CORRECTION_VERBATIM                          object
CUSTOMER_VERBATIM                            object
REPAIR_DATE                          datetime64[ns]
CAUSAL_PART_NM                               object
GLOBAL_LABOR_CODE_DESCRIPTION                object
PLATFORM                                     object
BODY_STYLE                                   object
VPPC                                         object
PLANT                                        object
BUILD_COUNTRY                                object
LAST_KNOWN_DLR_NAME                          object
LAST_KNOWN_DLR_CITY                          object
REPAIRING_DEALER_CODE                        object
DEALER_NAME                                  object
REPAIR_DLR_CITY                              object
STATE                                        object
DEALER_REGION             

In [23]:
def analyze_column_importance(df):
    """
    Analyze the importance of columns based on multiple metrics
    """
    # Initialize a DataFrame to store importance metrics
    importance_df = pd.DataFrame(index=df.columns)

    # Calculate missing value percentages
    importance_df["missing_percentage"] = df.isnull().mean() * 100

    # Calculate cardinality (number of unique values)
    importance_df["cardinality"] = df.apply(lambda x: x.nunique())
    importance_df["cardinality_ratio"] = importance_df["cardinality"] / len(df)

    # Calculate entropy for categorical columns
    def calculate_entropy(column):
        if column.dtype == "object":
            vc = column.value_counts(normalize=True, dropna=True)
            return stats.entropy(vc)
        return np.nan

    importance_df["entropy"] = df.apply(calculate_entropy)

    # Analyze text richness for text columns
    def text_richness(column):
        if column.dtype == "object":
            # Check if column contains text data
            sample = column.dropna().sample(min(100, len(column.dropna()))).astype(str)
            avg_words = sample.str.split().str.len().mean()
            return avg_words
        return np.nan

    importance_df["avg_word_count"] = df.apply(text_richness)

    # Calculate correlation with numerical target columns like REPAIR_AGE, TOTALCOST
    target_cols = ["REPAIR_AGE", "TOTALCOST", "LBRCOST"]
    for target in target_cols:
        if target in df.columns:
            # For numerical columns
            numerical_cols = df.select_dtypes(include=np.number).columns
            corr_dict = {}
            for col in numerical_cols:
                if col != target:
                    corr_dict[col] = df[[col, target]].corr().iloc[0, 1]

            # For categorical columns, use mutual information
            categorical_cols = df.select_dtypes(include=["object"]).columns
            for col in categorical_cols:
                if df[col].nunique() < 100:  # Skip high cardinality categorical
                    try:
                        # Convert to dummies for MI calculation
                        dummies = pd.get_dummies(df[col], drop_first=True)
                        X = (
                            dummies.values
                            if dummies.shape[1] > 0
                            else np.zeros((len(df), 1))
                        )
                        y = df[target].values
                        valid_mask = ~np.isnan(y)
                        if sum(valid_mask) > 0:
                            mi = mutual_info_regression(X[valid_mask], y[valid_mask])
                            corr_dict[col] = np.mean(mi)
                        else:
                            corr_dict[col] = np.nan
                    except:
                        corr_dict[col] = np.nan

            importance_df[f"corr_with_{target}"] = pd.Series(corr_dict)

    # Outlier percentage for numerical columns
    def outlier_percentage(column):
        if np.issubdtype(column.dtype, np.number):
            q1 = column.quantile(0.25)
            q3 = column.quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers = ((column < lower_bound) | (column > upper_bound)).sum()
            return (outliers / len(column)) * 100
        return np.nan

    importance_df["outlier_percentage"] = df.apply(outlier_percentage)

    return importance_df


# Calculate column importance
importance_results = analyze_column_importance(df)

# Add composite score based on all metrics
# Normalize each metric between 0 and 1
normalized_df = importance_results.copy()
for col in normalized_df.columns:
    if col != "missing_percentage" and col != "outlier_percentage":
        if normalized_df[col].notna().sum() > 0:
            min_val = normalized_df[col].min()
            max_val = normalized_df[col].max()
            if max_val > min_val:
                normalized_df[col] = (normalized_df[col] - min_val) / (
                    max_val - min_val
                )
    else:
        # For missing and outlier percentages, lower is better
        if normalized_df[col].notna().sum() > 0:
            min_val = normalized_df[col].min()
            max_val = normalized_df[col].max()
            if max_val > min_val:
                normalized_df[col] = 1 - (
                    (normalized_df[col] - min_val) / (max_val - min_val)
                )

# Calculate composite score (weighted average of normalized metrics)
weights = {
    "cardinality_ratio": 0.15,
    "entropy": 0.2,
    "avg_word_count": 0.2,
    "missing_percentage": 0.15,
    "outlier_percentage": 0.1,
}

# Add weights for correlation columns
corr_cols = [col for col in normalized_df.columns if col.startswith("corr_with_")]
if corr_cols:
    corr_weight = 0.2 / len(corr_cols)
    for col in corr_cols:
        weights[col] = corr_weight

# Calculate composite score
normalized_df["composite_score"] = 0
for col, weight in weights.items():
    if col in normalized_df.columns:
        normalized_df["composite_score"] += normalized_df[col].fillna(0) * weight

# Get top columns by composite score
top_columns = normalized_df.sort_values("composite_score", ascending=False)

print("Top 10 columns by importance score:")
print(top_columns["composite_score"].head(10))

# Extract top 5 critical columns
critical_columns = top_columns.index[:5].tolist()
print("\nIdentified 5 most critical columns:")
print(critical_columns)

# Detailed justification for each critical column
print("\nJustification for critical column selection:")
for col in critical_columns:
    print(f"\n{col}:")
    print(
        f"  - Missing values: {importance_results.loc[col, 'missing_percentage']:.2f}%"
    )
    if "avg_word_count" in importance_results.columns and not pd.isna(
        importance_results.loc[col, "avg_word_count"]
    ):
        print(
            f"  - Average word count: {importance_results.loc[col, 'avg_word_count']:.2f}"
        )
    if "cardinality" in importance_results.columns:
        print(f"  - Unique values: {importance_results.loc[col, 'cardinality']}")
    for target in ["REPAIR_AGE", "TOTALCOST", "LBRCOST"]:
        corr_col = f"corr_with_{target}"
        if corr_col in importance_results.columns and not pd.isna(
            importance_results.loc[col, corr_col]
        ):
            print(
                f"  - Correlation/MI with {target}: {importance_results.loc[col, corr_col]:.4f}"
            )
    print(
        f"  - Composite importance score: {normalized_df.loc[col, 'composite_score']:.4f}"
    )

Top 10 columns by importance score:
CORRECTION_VERBATIM      0.733256
CUSTOMER_VERBATIM        0.668245
VIN                      0.544583
LAST_KNOWN_DLR_CITY      0.540320
REPAIRING_DEALER_CODE    0.538206
REPAIR_DLR_CITY          0.537325
REPAIR_DLR_POSTAL_CD     0.528843
LAST_KNOWN_DLR_NAME      0.518129
DEALER_NAME              0.516282
ENGINE_TRACE_NBR         0.507318
Name: composite_score, dtype: float64

Identified 5 most critical columns:
['CORRECTION_VERBATIM', 'CUSTOMER_VERBATIM', 'VIN', 'LAST_KNOWN_DLR_CITY', 'REPAIRING_DEALER_CODE']

Justification for critical column selection:

CORRECTION_VERBATIM:
  - Missing values: 0.00%
  - Average word count: 18.32
  - Unique values: 93
  - Correlation/MI with REPAIR_AGE: 0.0121
  - Correlation/MI with TOTALCOST: 0.0043
  - Correlation/MI with LBRCOST: 0.0017
  - Composite importance score: 0.7333

CUSTOMER_VERBATIM:
  - Missing values: 0.00%
  - Average word count: 15.57
  - Unique values: 100
  - Composite importance score: 0.6682



In [24]:
def clean_data(df):
    """
    Clean the dataset by handling missing values, inconsistencies, and outliers
    """
    df_clean = df.copy()

    # Handle missing values
    numerical_cols = df_clean.select_dtypes(include=np.number).columns
    categorical_cols = df_clean.select_dtypes(include=["object"]).columns

    # For numerical columns, impute with median
    for col in numerical_cols:
        if df_clean[col].isnull().sum() > 0:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(
                f"Imputed {df_clean[col].isnull().sum()} missing values in {col} with median {median_val:.2f}"
            )

    # For categorical columns, impute with mode
    for col in categorical_cols:
        if df_clean[col].isnull().sum() > 0:
            mode_val = df_clean[col].mode()[0]
            df_clean[col] = df_clean[col].fillna(mode_val)
            print(
                f"Imputed {df_clean[col].isnull().sum()} missing values in {col} with mode '{mode_val}'"
            )

    # Standardize text in categorical columns (lowercase and trim)
    for col in categorical_cols:
        df_clean[col] = df_clean[col].astype(str).str.strip().str.lower()

    # Handle inconsistencies in categorical columns
    # For example, standardize common typos or variations
    # This would be more specific based on the actual data patterns

    # Handle outliers in numerical columns
    # Instead of removing, we'll create flags (already present as _OUTLIER columns)
    # We can use these flags in our analysis

    return df_clean


# Clean the data
df_clean = clean_data(df)

# Check the consistency of the data after cleaning
print("Dataset shape after cleaning:", df_clean.shape)
print("Missing values after cleaning:", df_clean.isnull().sum().sum())

Imputed 100 missing values in CAMPAIGN_NBR with median nan
Imputed 0 missing values in CAUSAL_PART_NM with mode 'WHEEL ASM-STRG *JET BLACK'
Imputed 0 missing values in PLANT with mode 'SIL'
Imputed 0 missing values in STATE with mode 'CA'
Imputed 0 missing values in REPAIR_DLR_POSTAL_CD with mode '907551909'
Imputed 0 missing values in VEH_TEST_GRP with mode 'T05.3386'
Imputed 0 missing values in OPTN_FAMLY_CERTIFICATION with mode 'FE9'
Imputed 0 missing values in OPTF_FAMLY_EMISSIOF_SYSTEM with mode 'FTB'
Imputed 0 missing values in ENGINE_SOURCE_PLANT with mode '830107152'
Imputed 0 missing values in ENGINE_TRACE_NBR with mode 'V2210281MFTX0488'
Imputed 0 missing values in TRANSMISSION_TRACE_NBR with mode '21210129IKBP0429'
Imputed 0 missing values in LINE_SERIES with mode '1500'
Dataset shape after cleaning: (100, 73)
Missing values after cleaning: 100


d:\#VSCode\Recommender\env\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\tirtn\AppData\Local\Temp\ipykernel_38328\4266995265.py:23: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  mode_val = df_clean[col].mode()[0]


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Ensure output directory exists
IMG_DIR = "img"
os.makedirs(IMG_DIR, exist_ok=True)


# 1. Visualize Column Importance
def visualize_column_importance(importance_df, top_n=10):
    """
    Create visualizations for column importance
    """
    # Horizontal bar chart for column importance
    plt.figure(figsize=(12, 8))
    top_cols = importance_df.sort_values("composite_score", ascending=False).head(top_n)
    sns.barplot(x=top_cols["composite_score"], y=top_cols.index)
    plt.title("Columns by Importance Score")
    plt.xlabel("Importance Score")
    plt.ylabel("Column Name")
    plt.tight_layout()
    plt.savefig(f"{IMG_DIR}/column_importance.png")
    plt.close()

    # Heatmap of important numerical columns
    num_cols = df.select_dtypes(include=np.number).columns
    top_num_cols = [col for col in top_cols.index if col in num_cols]
    if len(top_num_cols) > 1:
        plt.figure(figsize=(10, 8))
        correlation_matrix = df[top_num_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
        plt.title("Correlation Matrix of Important Numerical Columns")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/correlation_heatmap.png")
        plt.close()

    # Word clouds for key text columns
    text_cols = [
        "CUSTOMER_VERBATIM",
        "CORRECTION_VERBATIM",
        "GLOBAL_LABOR_CODE_DESCRIPTION",
    ]
    for col in text_cols:
        if col in df.columns:
            text = " ".join(df[col].dropna().astype(str))
            if len(text) > 100:
                plt.figure(figsize=(10, 8))
                wordcloud = WordCloud(
                    width=800,
                    height=800,
                    background_color="white",
                    max_words=200,
                    contour_width=3,
                    contour_color="steelblue",
                ).generate(text)
                plt.imshow(wordcloud, interpolation="bilinear")
                plt.axis("off")
                plt.title(f"Word Cloud - {col.replace('_', ' ')}")
                plt.tight_layout(pad=0)
                plt.savefig(f"{IMG_DIR}/wordcloud_{col.replace('_', ' ')}.png")
                plt.close()


# 2. Visualize Tag Distributions
def visualize_tag_distributions(df):
    """
    Visualize tag distributions (splitting comma-separated values)
    """
    tag_columns = [
        "ISSUES",
        "COMPONENTS",
        "ACTIONS",
        "REPAIR_COMPLEXITY",
        "VEHICLE_SYSTEM",
        "FAILURE_MODE",
        "REPAIR_URGENCY",
    ]

    for col in tag_columns:
        if col in df.columns:
            all_tags = (
                df[col]
                .dropna()
                .astype(str)
                .apply(lambda x: [tag.strip() for tag in x.split(",")])
            )
            flat_tags = [tag for sublist in all_tags for tag in sublist if tag]
            tag_counts = pd.Series(flat_tags).value_counts().nlargest(10)

            plt.figure(figsize=(12, 6))
            sns.barplot(x=tag_counts.values, y=tag_counts.index)
            plt.title(col.replace("_", " "))
            plt.xlabel("Count")
            plt.ylabel(col.replace("_", " "))
            plt.tight_layout(pad=2.0)
            plt.savefig(f"{IMG_DIR}/tag_distribution_{col}.png")
            plt.close()


# 3. Visualize Repair Patterns
def visualize_repair_patterns(df):
    """
    Visualize repair patterns and relationships
    """
    # Scatter plot: Repair Age vs Cost
    plt.figure(figsize=(10, 6))
    plt.scatter(df["REPAIR_AGE"], df["TOTALCOST"], alpha=0.5)
    plt.title("Repair Age vs Total Cost")
    plt.xlabel("Repair Age")
    plt.ylabel("Total Cost")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig(f"{IMG_DIR}/repair_age_vs_cost.png")
    plt.close()

    # Box plot: Cost by Repair Complexity
    if "REPAIR_COMPLEXITY" in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(x="REPAIR_COMPLEXITY", y="TOTALCOST", data=df)
        plt.title("Total Cost by Repair Complexity")
        plt.xlabel("Repair Complexity")
        plt.ylabel("Total Cost")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/cost_by_complexity.png")
        plt.close()

    # Box plot: Cost by Vehicle System
    if "VEHICLE_SYSTEM" in df.columns:
        top_systems = df["VEHICLE_SYSTEM"].value_counts().nlargest(5).index
        plt.figure(figsize=(12, 6))
        sns.boxplot(
            x="VEHICLE_SYSTEM",
            y="TOTALCOST",
            data=df[df["VEHICLE_SYSTEM"].isin(top_systems)],
        )
        plt.title("Total Cost by Vehicle System")
        plt.xlabel("Vehicle System")
        plt.ylabel("Total Cost")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/cost_by_system.png")
        plt.close()


# 4. Visualize Geographic Distribution
def visualize_geographic_distribution(df):
    """
    Visualize geographic distribution of repairs
    """
    if "STATE" in df.columns:
        # State-wise repair count
        plt.figure(figsize=(12, 6))
        state_counts = df["STATE"].value_counts().nlargest(10)
        sns.barplot(x=state_counts.index, y=state_counts.values)
        plt.title("State-wise Repair Count")
        plt.xlabel("State")
        plt.ylabel("Count")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/repairs_by_state.png")
        plt.close()

        # State-wise average cost
        state_avg_cost = df.groupby("STATE")["TOTALCOST"].mean().nlargest(10)
        plt.figure(figsize=(12, 6))
        sns.barplot(x=state_avg_cost.index, y=state_avg_cost.values)
        plt.title("State-wise Average Repair Cost")
        plt.xlabel("State")
        plt.ylabel("Average Cost")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/avg_cost_by_state.png")
        plt.close()


# 5. Visualize Platform and Engine Distribution
def visualize_platform_engine(df):
    """
    Visualize distribution of platform and engine types
    """
    # Platform distribution
    if "PLATFORM" in df.columns:
        plt.figure(figsize=(12, 6))
        platform_counts = df["PLATFORM"].value_counts().nlargest(10)
        sns.barplot(x=platform_counts.index, y=platform_counts.values)
        plt.title("Platform-wise Repair Count")
        plt.xlabel("Platform")
        plt.ylabel("Count")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/repairs_by_platform.png")
        plt.close()

    # Engine type distribution
    if "ENGINE" in df.columns:
        plt.figure(figsize=(12, 6))
        engine_counts = df["ENGINE"].value_counts().nlargest(10)
        sns.barplot(x=engine_counts.index, y=engine_counts.values)
        plt.title("Engine Type-wise Repair Count")
        plt.xlabel("Engine Type")
        plt.ylabel("Count")
        plt.xticks(rotation=0, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/repairs_by_engine.png")
        plt.close()


# Final Calls to Generate All Visualizations
visualize_column_importance(normalized_df)
visualize_tag_distributions(df_clean)
visualize_repair_patterns(df_clean)
visualize_geographic_distribution(df_clean)
visualize_platform_engine(df_clean)

In [ ]:
IMG_DIR = "img2"
os.makedirs(IMG_DIR, exist_ok=True)


def analyze_tag_relationships(df):
    """
    Analyze relationships between tags and create visualizations
    """
    # 1. Co-occurrence matrix for issues and components
    if all(col in df.columns for col in ["ISSUES", "COMPONENTS"]):
        # Split comma-separated values, flatten, and count
        issue_tags = (
            df["ISSUES"]
            .dropna()
            .astype(str)
            .apply(lambda x: [tag.strip() for tag in x.split(",")])
        )
        component_tags = (
            df["COMPONENTS"]
            .dropna()
            .astype(str)
            .apply(lambda x: [tag.strip() for tag in x.split(",")])
        )

        # Get top tags individually
        flat_issues = [tag for sublist in issue_tags for tag in sublist]
        flat_components = [tag for sublist in component_tags for tag in sublist]
        top_issues = pd.Series(flat_issues).value_counts().nlargest(10).index
        top_components = pd.Series(flat_components).value_counts().nlargest(10).index

        # Initialize co-occurrence matrix
        cooccurrence = pd.DataFrame(index=top_issues, columns=top_components, data=0)

        for i, row in df.iterrows():
            row_issues = [tag.strip() for tag in str(row.get("ISSUES", "")).split(",")]
            row_components = [
                tag.strip() for tag in str(row.get("COMPONENTS", "")).split(",")
            ]
            for issue in row_issues:
                for component in row_components:
                    if issue in top_issues and component in top_components:
                        cooccurrence.loc[issue, component] += 1

        # Create heatmap
        plt.figure(figsize=(12, 10))
        sns.heatmap(cooccurrence, annot=True, cmap="YlGnBu", fmt="d")
        plt.title("Co-occurrence of Issues and Components")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/issue_component_cooccurrence.png")
        plt.close()

    # 2. Average repair cost by component and complexity
    if all(
        col in df.columns for col in ["COMPONENTS", "REPAIR_COMPLEXITY", "TOTALCOST"]
    ):
        component_tags = (
            df["COMPONENTS"]
            .dropna()
            .astype(str)
            .apply(lambda x: [tag.strip() for tag in x.split(",")])
        )
        flat_components = [tag for sublist in component_tags for tag in sublist]
        top_components = pd.Series(flat_components).value_counts().nlargest(8).index

        # Expand rows for multiple components
        expanded_rows = []
        for _, row in df.iterrows():
            components = [
                tag.strip() for tag in str(row.get("COMPONENTS", "")).split(",")
            ]
            for comp in components:
                if comp in top_components:
                    expanded_rows.append(
                        {
                            "COMPONENTS": comp,
                            "REPAIR_COMPLEXITY": row["REPAIR_COMPLEXITY"],
                            "TOTALCOST": row["TOTALCOST"],
                        }
                    )
        cost_df = pd.DataFrame(expanded_rows)

        # Plot grouped bar chart
        plt.figure(figsize=(14, 8))
        sns.barplot(
            x="COMPONENTS", y="TOTALCOST", hue="REPAIR_COMPLEXITY", data=cost_df
        )
        plt.title("Average Repair Cost by Component and Complexity")
        plt.xlabel("Component")
        plt.ylabel("Average Cost")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/cost_by_component_complexity.png")
        plt.close()

    # 3. Relationship between repair age and repair complexity
    if all(col in df.columns for col in ["REPAIR_AGE", "REPAIR_COMPLEXITY"]):
        plt.figure(figsize=(10, 6))
        sns.boxplot(x="REPAIR_COMPLEXITY", y="REPAIR_AGE", data=df)
        plt.title("Repair Age by Complexity")
        plt.xlabel("Repair Complexity")
        plt.ylabel("Repair Age")
        plt.tight_layout()
        plt.savefig(f"{IMG_DIR}/repair_age_by_complexity.png")
        plt.close()


# Call to run
analyze_tag_relationships(df_clean)

In [ ]:
def generate_insights_summary(df, importance_df, critical_columns):
    """
    Generate insights and summary from the analysis
    """
    insights = []

    # 1. Critical columns insights
    insights.append("=== Critical Columns Analysis ===")
    insights.append(
        f"The top 5 most important columns identified are: {', '.join(critical_columns)}"
    )

    for col in critical_columns:
        score = (
            importance_df.loc[col, "composite_score"]
            if "composite_score" in importance_df.columns
            else "N/A"
        )
        if isinstance(score, (int, float)):
            insights.append(f"- {col.replace('_', ' ')}: Importance score: {score:.4f}")
        else:
            insights.append(f"- {col.replace('_', ' ')}: Importance score: {score}")

        if col in ["CUSTOMER_VERBATIM", "CORRECTION_VERBATIM"]:
            insights.append(
                "  - Contains rich textual information about repair issues and actions"
            )
            if "avg_word_count" in importance_df.columns:
                insights.append(
                    f"  - Average word count: {importance_df.loc[col, 'avg_word_count']:.2f}"
                )
        elif col in ["REPAIR_AGE", "KM", "TOTALCOST", "LBRCOST"]:
            insights.append(
                "  - Critical metric for repair analysis and cost assessment"
            )
            for target in ["REPAIR_AGE", "TOTALCOST", "LBRCOST"]:
                key = f"corr_with_{target}"
                if target != col and key in importance_df.columns:
                    insights.append(
                        f"  - Correlation with {target.replace('_', ' ')}: {importance_df.loc[col, key]:.4f}"
                    )

    # 2. Tag distribution insights
    insights.append("\n=== Tag Analysis ===")
    tag_columns = [
        "ISSUES",
        "COMPONENTS",
        "ACTIONS",
        "REPAIR_COMPLEXITY",
        "VEHICLE_SYSTEM",
        "FAILURE_MODE",
        "REPAIR_URGENCY",
    ]

    for col in tag_columns:
        if col in df.columns:
            all_tags = (
                df[col]
                .dropna()
                .astype(str)
                .apply(lambda x: [tag.strip() for tag in x.split(",")])
            )
            flat_tags = [tag for sublist in all_tags for tag in sublist if tag]
            top_values = pd.Series(flat_tags).value_counts().nlargest(3)
            insights.append(f"- Top {col.replace('_', ' ')}:")
            for val, count in top_values.items():
                insights.append(
                    f"  - {val}: {count} occurrences ({count / len(df) * 100:.1f}%)"
                )

    # 3. Cost analysis insights
    insights.append("\n=== Cost Analysis ===")
    if "TOTALCOST" in df.columns:
        insights.append(f"- Average total cost: ${df['TOTALCOST'].mean():.2f}")
        insights.append(f"- Median total cost: ${df['TOTALCOST'].median():.2f}")
        insights.append(
            f"- Cost range: ${df['TOTALCOST'].min():.2f} to ${df['TOTALCOST'].max():.2f}"
        )

    if "LBRCOST" in df.columns and "TOTALCOST" in df.columns:
        labor_ratio = (df["LBRCOST"] / df["TOTALCOST"]).replace(
            [np.inf, -np.inf], np.nan
        ).dropna().mean() * 100
        insights.append(f"- Average labor cost ratio: {labor_ratio:.1f}% of total cost")

    if "REPAIR_COMPLEXITY" in df.columns and "TOTALCOST" in df.columns:
        complexity_cost = (
            df.groupby("REPAIR_COMPLEXITY")["TOTALCOST"]
            .mean()
            .sort_values(ascending=False)
        )
        insights.append("- Average cost by complexity:")
        for complexity, cost in complexity_cost.items():
            insights.append(f"  - {complexity}: ${cost:.2f}")

    # 4. Vehicle system insights
    insights.append("\n=== Vehicle System Analysis ===")
    if "VEHICLE_SYSTEM" in df.columns:
        system_counts = df["VEHICLE_SYSTEM"].value_counts().nlargest(5)
        insights.append("- Most common vehicle systems requiring repair:")
        for system, count in system_counts.items():
            insights.append(
                f"  - {system}: {count} repairs ({count / len(df) * 100:.1f}%)"
            )

    if "VEHICLE_SYSTEM" in df.columns and "TOTALCOST" in df.columns:
        system_costs = df.groupby("VEHICLE_SYSTEM")["TOTALCOST"].mean().nlargest(5)
        insights.append("- Most expensive vehicle systems to repair:")
        for system, cost in system_costs.items():
            insights.append(f"  - {system}: ${cost:.2f} average cost")

    # 5. Geographic insights
    insights.append("\n=== Geographic Analysis ===")
    if "STATE" in df.columns:
        state_counts = df["STATE"].value_counts().nlargest(5)
        insights.append("- States with most repairs:")
        for state, count in state_counts.items():
            insights.append(
                f"  - {state}: {count} repairs ({count / len(df) * 100:.1f}%)"
            )

        if "TOTALCOST" in df.columns:
            state_costs = df.groupby("STATE")["TOTALCOST"].mean().nlargest(5)
            insights.append("- States with highest average repair costs:")
            for state, cost in state_costs.items():
                insights.append(f"  - {state}: ${cost:.2f} average cost")

    # Print all insights
    print("\n".join(insights))


# Generate insights summary
generate_insights_summary(df_clean, normalized_df, critical_columns)

=== Critical Columns Analysis ===
The top 5 most important columns identified are: CORRECTION_VERBATIM, CUSTOMER_VERBATIM, VIN, LAST_KNOWN_DLR_CITY, REPAIRING_DEALER_CODE
- CORRECTION VERBATIM: Importance score: 0.7333
  - Contains rich textual information about repair issues and actions
  - Average word count: 1.00
- CUSTOMER VERBATIM: Importance score: 0.6682
  - Contains rich textual information about repair issues and actions
  - Average word count: 0.84
- VIN: Importance score: 0.5446
- LAST KNOWN DLR CITY: Importance score: 0.5403
- REPAIRING DEALER CODE: Importance score: 0.5382

=== Tag Analysis ===
- Top ISSUES:
  - material problems: 30 occurrences (30.0%)
  - heating malfunctions: 24 occurrences (24.0%)
  - stitching failures: 11 occurrences (11.0%)
- Top COMPONENTS:
  - steering wheel (main component): 93 occurrences (93.0%)
  - leather/material covering: 23 occurrences (23.0%)
  - heated module: 21 occurrences (21.0%)
- Top ACTIONS:
  - part replacement: 91 occurrences (91